In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
!nvidia-smi # test we are on GPU

Thu Nov 20 20:10:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%pip uninstall -y torch torchaudio torchvision

%pip install torch==2.9.1 --index-url https://download.pytorch.org/whl/test/cu126 -q # 2.9.1 has Muon
%pip install torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/test/cu126 -q 

%pip install torchvision==0.24.1+cu126 --index-url https://download.pytorch.org/whl/cu126 -q


# above must be removed in kaggle
%pip install timm wandb==0.22.0 torchmetrics numpy tensorboard matplotlib -q #--no-deps -q # no-deps to avoid torch 2.8.0, but wandb needs newer vers

# alt vers for kaggle ?

# %pip install --quiet torch==2.9.1+cu126 torchvision==0.24.1+cu126 torchaudio==2.9.1+cu126 --index-url https://download.pytorch.org/whl/test/cu126

# %pip install --quiet timm==0.9.2 torchmetrics==0.11.4 matplotlib==3.8.1

# %pip install --quiet wandb==0.22.0

# %pip install --quiet tensorboard==2.13.0 protobuf==4.23.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.22 requires torchvision, which is not installed.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 85.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 110.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 67.9 MB/s eta 0:00:00


In [4]:
import os

import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision.transforms import v2
from torch.backends import cudnn
from torch import GradScaler
from torch import optim
from tqdm import tqdm
import numpy as np
import pickle
import time

from torchvision.datasets import CIFAR10, CIFAR100, MNIST, OxfordIIITPet
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import timm
import wandb
wandb.login(key=os.getenv('WANDB_API_KEY')) # kaggle uses kaggle_secrets, here use .env
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter



/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
print("Muon" in dir(torch.optim))
print(torch.__version__)

In [ ]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else torch.device("cpu")
enable_half = device.type != "cpu"
# scaler = torch.cuda.amp.GradScaler(device, enabled=enable_half)

print("Grad scaler is enabled:", enable_half)
print("Device:", device)

In [ ]:

batch_size_map = {
    "resnet18": 128,
    "resnet50": 64,
    "resnest14d": 32,
    "resnest26d": 16,
    "mlp": 256
}   

# # placeholder, gets overwritten by sweep_config in last cell
# config = {
#     "run_name": "cifar100_resnet18_amp",
#     "dataset": "CIFAR100",
#     "model": "resnet18",
#     "epochs": 50,
#     "lr": 1e-1,
#     "weight_decay": 1e-4,
#     "batch_size": batch_size["resnet18"],
#     "pretrained": False,
#     "optimizer": "AdamW",
#     "scheduler": "StepLR",
#     "image_size": 32, # 224 for pretrained True
#     "num_workers": 2,
#     "pin_memory": True,
#     "use_amp": True,
#     "use_wandb": True
# }



In [ ]:
class BatchSizeScheduler:
    def __init__(self, initial_batch_size, max_batch_size, step_size=30):
        self.initial_batch_size = initial_batch_size
        self.current_batch_size = initial_batch_size
        self.max_batch_size = max_batch_size
        self.step_size = step_size
        self.epoch = 0
    
    def step(self):
        self.epoch += 1
        if self.epoch % self.step_size == 0 and self.current_batch_size < self.max_batch_size:
            old_bs = self.current_batch_size
            self.current_batch_size = min(int(self.current_batch_size * 1.5), self.max_batch_size)
            if self.current_batch_size != old_bs:
                print(f"Batch size increased: {old_bs} → {self.current_batch_size}")
                return True
        return False
    
    def get_batch_size(self):
        return self.current_batch_size

In [ ]:
def get_transforms(dataset, image_size, is_train=True):
    """Get appropriate transforms based on dataset and image size"""
    
    if dataset == "MNIST":
        return v2.Compose([
            v2.ToImage(),
            v2.Resize(image_size),
            v2.ToDtype(torch.float32, scale=True),
        ])
    
    if dataset in ["CIFAR10", "CIFAR100"]:
        mean = (0.5071, 0.4867, 0.4408)
        std = (0.2675, 0.2565, 0.2761)
    else:  # OxfordIIITPet
        mean = (0.485, 0.456, 0.406)
        std = (0.229, 0.224, 0.225)
    
    if is_train:
        if image_size == 32:
            return v2.Compose([
                v2.ToImage(),
                v2.RandomCrop(32, padding=4),
                v2.RandomHorizontalFlip(),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(mean, std)
            ])
        else:
            return v2.Compose([
                v2.ToImage(),
                v2.Resize(256),
                v2.RandomCrop(image_size),
                v2.RandomHorizontalFlip(),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(mean, std)
            ])
    else: #test
        return v2.Compose([
            v2.ToImage(),
            v2.Resize(image_size),
            v2.CenterCrop(image_size),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean, std)
        ])

# DATA_ROOT = "kaggle/working/data"
# os.makedirs('./data', exist_ok=True)

# in kaggle use DATA_ROOT and replace root='./data' 
# also replace download with =not os.path.exists(f"{DATA_ROOT}/....")


def get_data_loaders(dataset_name, image_size, batch_size, num_workers=2, pin_memory=True):
    train_transforms = get_transforms(dataset_name, image_size, is_train=True)
    test_transform = get_transforms(dataset_name, image_size, is_train=False)

    if dataset_name == "CIFAR100":
        train_dataset= CIFAR100(root='./data', train=True, download=True, transform=train_transforms)
        test_dataset = CIFAR100(root='./data', train=False, download=True, transform=test_transform)
    elif dataset_name == "CIFAR10":
        train_dataset= CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=test_transform)
    elif dataset_name == "MNIST":
        train_dataset= MNIST(root='./data', train=True, download=True, transform=train_transforms)
        test_dataset = MNIST(root='./data', train=False, download=True, transform=test_transform)
    elif dataset_name == "OxfordIIITPet":
        train_dataset= OxfordIIITPet(root='./data', download=True, transform=train_transforms)
        test_dataset = OxfordIIITPet(root='./data', split='test', download=True, transform=test_transform)
    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
    return train_loader, test_loader

In [ ]:
def get_num_classes(dataset_name):
    if dataset_name in ["CIFAR100"]:
        return 100
    elif dataset_name in ["CIFAR10"]:
        return 10
    elif dataset_name in ["MNIST"]:
        return 10
    elif dataset_name in ["OxfordIIITPet"]:
        return 37
    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")

In [ ]:
# if config["model"] == "mlp":
#     model = nn.Sequential(
#         nn.Flatten(),
#         nn.Linear(3 * config["image_size"] * config["image_size"], 512),
#         nn.ReLU(),
#         nn.Linear(512, 256), # num_classes
#         nn.ReLU(),
#         nn.Linear(256, 100)
#     )
# else:
#     model = timm.create_model(config["model"], pretrained=config["pretrained"], num_classes=100)
#     # whats this for 
#     model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
#     model.maxpool = nn.Identity()

# model.to(device)

In [ ]:
ALLOWED_MODELS = ["resnet18", "resnet50", "resnest14d", "resnest26d", "mlp"]

def create_model(model_name, dataset_name, pretrained=False, image_size=32):
    if model_name not in ALLOWED_MODELS:
        raise ValueError(f"Unsupported model: {model_name}")
    
    num_classes=get_num_classes(dataset_name)

    if model_name == "mlp":
        in_channels = 1 if dataset_name == "MNIST" else 3
        return nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_channels * image_size * image_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    else:
        model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)
        if not pretrained and image_size == 32:
            if model_name.startswith("resnet") or model_name.startswith("resnest"):
                model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
                model.maxpool = nn.Identity()
    return model

In [ ]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"
        self.rho = rho
        self.adaptive = adaptive

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        # Base optimizer instance
        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()

        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None:
                    continue
                e = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale
                p.add_(e)  # ascent step
                self.state[p]["e"] = e

        if zero_grad:
            self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                p.sub_(self.state[p]["e"])  # restore weights

        self.base_optimizer.step()

        if zero_grad:
            self.zero_grad()

    def zero_grad(self):
        self.base_optimizer.zero_grad()

    def step(self):
        # Not used — SAM requires first_step + second_step manually
        raise RuntimeError("Use first_step() and second_step() with SAM.")
    
    def _grad_norm(self):
        norm = torch.norm(
            torch.stack([
                ((torch.pow(p, 2) if self.adaptive else 1.0) * p.grad).norm(p=2)
                for group in self.param_groups for p in group["params"]
                if p.grad is not None
            ]),
            p=2
        )
        return norm


# lr = config["lr"] 
# wd = config["weight_decay"]

# if config["optimizer"] == "SGD":
#     optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
# elif config["optimizer"] == "AdamW":
#     optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
# elif config["optimizer"] == "Adam":
#     optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
# elif config["optimizer"] == "SAM":
#     optimizer = SAM(model.parameters(), base_optimizer=optim.SGD, lr=lr, momentum=0.9, weight_decay=wd)
# elif config["optimizer"] == "Muon":
#     optimizer = torch.optim.Muon(model.parameters(), lr=lr, weight_decay=wd)        
#     # add sam muon

def create_optimizer(optimizer_name, model_parameters, lr, weight_decay):
    if optimizer_name == "SGD":
        return optim.SGD(model_parameters, lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        return optim.AdamW(model_parameters, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "Adam":
        return optim.Adam(model_parameters, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SAM":
        return SAM(model_parameters, base_optimizer=optim.SGD, lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "Muon":
        return torch.optim.Muon(model_parameters, lr=lr, weight_decay=weight_decay)        
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

In [ ]:
# if config["scheduler"] == "StepLR":
#     scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
# elif config["scheduler"] == "ReduceLROnPlateau":
#     scheduler = ReduceLROnPlateau(optimizer, patience=5)

def create_scheduler(scheduler_name, optimizer):
    base_optimizer = optimizer.base_optimizer if isinstance(optimizer, SAM) else optimizer

    if scheduler_name == "StepLR":
        return StepLR(base_optimizer, step_size=30, gamma=0.1)
    elif scheduler_name == "ReduceLROnPlateau":
        return ReduceLROnPlateau(base_optimizer, patience=5)
    else:
        raise ValueError(f"Unsupported scheduler: {scheduler_name}")

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, scaler, device, use_amp):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    i = 0
    for inputs, targets in train_loader:
        i += 1
        inputs, targets = inputs.to(device), targets.to(device)


        with torch.amp.autocast(device_type=device.type, enabled=use_amp):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        if isinstance(optimizer, SAM):
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer.base_optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.first_step(zero_grad=True)

            with torch.amp.autocast(device_type=device.type, enabled=use_amp):
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer.base_optimizer)
            scaler.step(optimizer.base_optimizer)
            scaler.update()
            optimizer.zero_grad()
        else: 
            # if (i + 1) % 1 == 0:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update() 
            optimizer.zero_grad()


        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return train_loss / len(train_loader), 100.0 * correct/total


def test(model, test_loader, criterion, device, use_amp):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            with torch.amp.autocast(device_type=device.type, enabled=use_amp):
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return test_loss / len(test_loader), 100.0 * correct/total

def train_model(config):
    criterion = nn.CrossEntropyLoss()
    scaler = torch.amp.GradScaler(device.type, enabled=config["use_amp"])
    writer = SummaryWriter(log_dir=f'./logs/{config["dataset"]}/{config["optimizer"]}_{config["model"]}_bs{batch_size_map[config["model"]]}_lr{config["lr"]}_sched{config["scheduler"]}')

    image_size = 224 if config["pretrained"] else 32
    initial_batch_size = batch_size_map[config["model"]]
    
    use_bs_scheduler = config.get("use_batch_size_scheduler", False)
    if use_bs_scheduler:
        bs_scheduler = BatchSizeScheduler(
            initial_batch_size=initial_batch_size // 2, 
            max_batch_size=initial_batch_size,
            step_size=30
        )
        batch_size = bs_scheduler.get_batch_size()
        print(f"Batch size scheduler enabled: {batch_size} → {initial_batch_size}")
    else:
        batch_size = initial_batch_size

    train_loader, test_loader = get_data_loaders(config["dataset"], image_size, batch_size, config["num_workers"], config["pin_memory"]) 
    model = create_model(config["model"], config["dataset"], config["pretrained"], image_size)
    model.to(device)

    optimizer = create_optimizer(config["optimizer"], model.parameters(), config["lr"], config["weight_decay"])
    scheduler = create_scheduler(config["scheduler"], optimizer)

    epochs = config["epochs"]
    
    best_acc = 0.0

    best_acc_epoch = None
    best_loss = None
    counter = 0
    patience = 10

    start_time = time.time()
    print("Start training")
    
    for epoch in range(epochs):
        epoch_start = time.time()


        if use_bs_scheduler and bs_scheduler.step():
            batch_size = bs_scheduler.get_batch_size()
            train_loader, test_loader = get_data_loaders(
                config["dataset"], 
                image_size, 
                batch_size,
                config.get("num_workers", 2),
                config.get("pin_memory", True)
            )

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler, device, config["use_amp"])
        test_loss, test_acc = test(model, test_loader, criterion, device, config["use_amp"])

        if isinstance(scheduler, StepLR):
            scheduler.step()
        elif isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(test_loss)
        
        if isinstance(optimizer, SAM):
            current_lr = optimizer.base_optimizer.param_groups[0]['lr']
        else:
            current_lr = optimizer.param_groups[0]['lr']
        
        epoch_time = time.time() - epoch_start

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%, LR: {current_lr:.6f}, Time: {epoch_time:.2f}s")
        
        if config["use_wandb"]:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": test_loss,
                "val_acc": test_acc,
                "lr": current_lr,
                "epoch_time": epoch_time,
                "batch_size": batch_size if use_bs_scheduler else initial_batch_size
            })

        writer.add_scalar('Loss/Train', train_loss, epoch)
        writer.add_scalar('Loss/Test', test_loss, epoch)
        writer.add_scalar('Accuracy/Train', train_acc, epoch)
        writer.add_scalar('Accuracy/Test', test_acc, epoch)
        writer.add_scalar('Learning Rate', current_lr, epoch)
        if use_bs_scheduler:
            writer.add_scalar('Batch_Size', batch_size, epoch)
        

        if test_acc > best_acc:
            best_acc = test_acc
            os.makedirs('./checkpoints', exist_ok=True)

            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_acc': best_acc,    
                'config': config    
            }, f'./checkpoints/{config["optimizer"]}_{config["model"]}_bs{batch_size_map[config["model"]]}_lr{config["lr"]}_sched{config["scheduler"]}.pth')
            print(f"Best model saved with accuracy: {best_acc:.2f}%")

        # early stop mechanism
        if best_acc_epoch is None:
            best_acc_epoch = test_acc
        elif test_acc < best_acc_epoch:
            counter += 1
            if counter >= patience:
                print(f"Early stopping triggered for epoch {epoch+1}")
                break
        else:
            best_acc_epoch = test_acc
            counter = 0

    total_time = time.time() - start_time
    print(f"Training completed in {total_time/60:.2f} minutes")
    print(f"Best Test Accuracy: {best_acc:.2f}%")
    
    writer.close()

    if config["use_wandb"]:
            wandb.log({
                "epoch": epoch + 1,
                "best_acc": best_acc,
                "train_acc": train_acc,
                "lr": current_lr,
                "epoch_time": epoch_time,
                "batch_size": batch_size if use_bs_scheduler else initial_batch_size
            })

    return {
        'best_acc': best_acc,
        'total_time': total_time,
        'final_train_acc': train_acc,
        'final_test_acc': test_acc
    }


In [ ]:
def create_sweep_config(pretrained=False):
    if pretrained:
        return {
            "method": "grid",
            "metric": {"name": "val_acc", "goal": "maximize"},
            "parameters": {
                "dataset": {"value": "CIFAR100"},
                "model": {"values": ["resnet18", "resnet50"]},
                "pretrained": {"value": True}, 
                "lr": {"values": [3e-4, 1e-4]},
                "optimizer": {"value": "AdamW"},  
                "scheduler": {"value": "StepLR"},  
                "epochs": {"value": 100},
                "weight_decay": {"value": 1e-2},
                "num_workers": {"value": 2},  
                "pin_memory": {"value": True},
                "use_amp": {"value": True},  
                "use_wandb": {"value": True},
                "use_batch_size_scheduler": {"value": False}
            }
        }
    else:
        return {
            "method": "grid",
            "metric": {"name": "val_acc", "goal": "maximize"},
            "parameters": {
                "dataset": {"value": "CIFAR100"},
                "model": {"values": ["resnet18", "resnet50"]},
                "pretrained": {"value": False}, 
                "lr": {"values": [0.1, 0.05]},  
                "optimizer": {"value": "SGD"}, 
                "scheduler": {"values": ["StepLR", "ReduceLROnPlateau"]},
                "epochs": {"value": 200},
                "weight_decay": {"value": 5e-4},
                "num_workers": {"value": 2}, 
                "pin_memory": {"value": True},
                "use_amp": {"value": True}, 
                "use_wandb": {"value": True},
                "use_batch_size_scheduler": {"value": False}
            }
        }

In [ ]:
def sweep_train():
    run = wandb.init(project="training_pipeline")

    cfg = dict(wandb.config)
    results = train_model(cfg)

    # if cfg["model"] == "mlp":
    #     model = nn.Sequential(
    #         nn.Flatten(),
    #         nn.Linear(3 * cfg["image_size"] * cfg["image_size"], 512),
    #         nn.ReLU(),
    #         nn.Linear(512, 256), # num_classes
    #         nn.ReLU(),
    #         nn.Linear(256, 100)
    #     )
    # else:
    #         model = timm.create_model(cfg["model"], pretrained=cfg["pretrained"], num_classes=100)
    #         # whats this for 
    #         if not cfg["pretrained"]:
    #             model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    #             model.maxpool = nn.Identity()

    # model.to(device)
    # lr = cfg["lr"] 
    # wd = cfg["weight_decay"]

    # if cfg["optimizer"] == "SGD":
    #     optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
    # elif cfg["optimizer"] == "AdamW":
    #     optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    # elif cfg["optimizer"] == "Adam":
    #     optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    # elif cfg["optimizer"] == "SAM":
    #     optimizer = SAM(model.parameters(), base_optimizer=optim.SGD, lr=lr, momentum=0.9, weight_decay=wd)
    # elif cfg["optimizer"] == "Muon":
    #     optimizer = torch.optim.Muon(model.parameters(), lr=lr, weight_decay=wd)        
    #     # add sam muon

    # if cfg["scheduler"] == "StepLR":
    #     scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
    # elif cfg["scheduler"] == "ReduceLROnPlateau":
    #     scheduler = ReduceLROnPlateau(optimizer, patience=5)

    # results = train_model(train_loader, test_loader, model, optimizer, scheduler, cfg["epochs"], cfg["batch_size"], cfg["lr"])
    run.finish()
    

In [ ]:
sweep_config = create_sweep_config(pretrained=False)
sweep_id = wandb.sweep(sweep_config, project="training_pipeline")
wandb.agent(sweep_id, function=sweep_train)